<h1>Testes com os dados do Tilecal</h1>

Clonando repositório para ter acesso aos dados

In [12]:
!git clone https://github.com/carloscdias/nn-tilecal.git

fatal: destination path 'nn-tilecal' already exists and is not an empty directory.


Leitura dos dados do arquivo texto para um array de arrays em python.

In [13]:
# leitura do arquivo
with open('./nn-tilecal/data/pulses_uni0-1024_and_noise_zbEBAch01mu40.txt', 'r') as file:
  tilecal_data = [[int(value) for value in line.split()] for line in file.readlines()]

print("Arquivo contendo {} linhas com {} valores por linha".format(len(tilecal_data), len(tilecal_data[0])))

Arquivo contendo 304321 linhas com 8 valores por linha


Normalizando e separando dataset entre treino e teste: 70% treino, 30% teste

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np_arr = np.array(tilecal_data)
X, y = np_arr[:, :7].astype('float32'), np_arr[:, 7].astype('float32')

X_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(y.reshape(len(y), 1))
X = X_scaler.transform(X)
y = y_scaler.transform(y.reshape(len(y), 1))[:,0]

# 50% dos dados serão usados para treinamento do modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
# os 50% restantes serão divididos igualmente (25%, 25%)
# para serem usados como conjuntos de teste e validação
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5)

Construindo, treinando e avaliando o modelo

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=7))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

opt = SGD(lr=0.01, momentum=0.9)

model.compile(optimizer=opt,
              loss='mean_squared_error',
              metrics=['mse', 'mae', 'mape', 'cosine'])

history = model.fit(X_train, y_train,
          batch_size=50, epochs=5,
          verbose=1, validation_data=(X_valid, y_valid))

score = model.evaluate(X_test, y_test, verbose=0)

print(score)

Train on 152160 samples, validate on 76081 samples
Epoch 1/5
152160/152160 [==============================] - 4s 27us/step - loss: 0.0119 - mean_squared_error: 0.0119 - mean_absolute_error: 0.0715 - mean_absolute_percentage_error: 37.5554 - cosine_proximity: -0.9559 - val_loss: 0.0086 - val_mean_squared_error: 0.0086 - val_mean_absolute_error: 0.0618 - val_mean_absolute_percentage_error: 32.0179 - val_cosine_proximity: -0.9614
Epoch 2/5
152160/152160 [==============================] - 4s 25us/step - loss: 0.0078 - mean_squared_error: 0.0078 - mean_absolute_error: 0.0610 - mean_absolute_percentage_error: 31.6898 - cosine_proximity: -0.9632 - val_loss: 0.0073 - val_mean_squared_error: 0.0073 - val_mean_absolute_error: 0.0607 - val_mean_absolute_percentage_error: 30.8607 - val_cosine_proximity: -0.9635
Epoch 3/5
152160/152160 [==============================] - 4s 25us/step - loss: 0.0067 - mean_squared_error: 0.0067 - mean_absolute_error: 0.0565 - mean_absolute_percentage_error: 28.8140 -

Criando modelo atual do tilecal (OF2) para fins comparativos

In [0]:
of2_weights = np.array([-0.3781, -0.3572, 0.1808, 0.8125, 0.2767, -0.2056, -0.3292])
of2 = lambda x: np.inner(x, of2_weights)

Calculando o desvio padrão das distribuições dada por:

O_model = A_reco - A_true

O_of2 = A_of2 - A_true

In [17]:
import pandas as pd

A_true = y_test
A_reco = model.predict(X_test).reshape(-1)
A_of2  = np.array([of2(x) for x in X_test])

O_model = A_reco - A_true
O_of2   = A_of2 - A_true

s_model = pd.Series(O_model.tolist())
s_of2   = pd.Series(O_of2.tolist())

print("Desvio padrão modelo:", s_model.std())
print("Desvio padrão OF2:", s_of2.std())

Desvio padrão modelo: 0.06872537765754318
Desvio padrão OF2: 0.7845358097274902
